In [15]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
from functools import partial
import sys
sys.path.insert(0, '..')
import harmonic as hm
sys.path.append("../examples")
import utils

In [16]:
def ln_prior_uniform(x, xmin=-6.0, xmax=6.0, ymin=-6.0, ymax=6.0):
    if x[0] >= xmin and x[0] <= xmax and x[1] >= ymin and x[1] <= ymax:        
        return 1.0 / ( (xmax - xmin) * (ymax - ymin) )
    else:
        return 0.0

In [6]:
# def ln_likelihood(x):
#     f = (x[0]**2 + x[1] - 11.0)**2 + (x[0] + x[1]**2 - 7.0)**2
#     return -f
def ln_likelihood(x):
    """Compute log_e of likelihood defined by Rastrigin function.

    Args:

        x: Position at which to evaluate likelihood.

    Returns:

        double: Value of Rastrigin at specified point.

    """
    
    ndim = x.size

    f = 10.0 * ndim

    for i_dim in range(ndim):
        f += x[i_dim]**2 - 10.0 * np.cos( 2.0 * np.pi * x[i_dim] )

    return -f

In [7]:
def ln_posterior(x, ln_prior):
    ln_L = ln_likelihood(x)

    if not np.isfinite(ln_L):
        return -np.inf
    else:
        return ln_prior(x) + ln_L

In [8]:
# Define parameters for emcee sampling
ndim = 2                    # number of dimensions
nchains = 200               # total number of chains to compute
samples_per_chain = 5000    # number of samples per chain
nburn = 2000                # number of samples to discard as burn in

# Initialize random seed
np.random.seed(4)

# Define ln_prior function
xmin = -6.0
xmax = 6.0
ymin = -6.0
ymax = 6.0  
ln_prior = partial(ln_prior_uniform, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)  

In [9]:
# Set initial random position and state
pos = np.random.rand(ndim * nchains).reshape((nchains, ndim)) * 0.5  
rstate = np.random.get_state()

# Instantiate and execute sampler 
sampler = emcee.EnsembleSampler(nchains, ndim, ln_posterior, args=[ln_prior])
(pos, prob, state) = sampler.run_mcmc(pos, samples_per_chain, rstate0=rstate) 

# Collect samples into contiguous numpy arrays (discarding burn in)
samples = np.ascontiguousarray(sampler.chain[:,nburn:,:])
lnprob = np.ascontiguousarray(sampler.lnprobability[:,nburn:])

In [10]:
# Instantiate harmonic's chains class 
chains = hm.Chains(ndim)
chains.add_chains_3d(samples, lnprob)

In [11]:
# Split the chains into the ones which will be used to train the machine 
# learning model and for inference
chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.5)

In [12]:
# Define Kernel Density Estimate (KDE) hyperparameters
nfold = 2
nhyper = 2
step = -2
domains_KDE = [] # no defined domain for KDE estimate
hyper_parameters_KDE = [[10**(R)] for R in range(-nhyper+step,step)] 

# Define Hypersphere (sphere) hyperparameters
hyper_parameters_sphere = [None]
domains_sphere = [np.array([1E-2,1E1])]

In [13]:
validation_variances_KDE = \
            hm.utils.cross_validation(
                    chains_train, \
                    domains_KDE, \
                    hyper_parameters_KDE, \
                    nfold=nfold, \
                    modelClass=hm.model.KernelDensityEstimate, \
                    seed=0)
best_hyper_param_ind_KDE = np.argmin(validation_variances_KDE)
best_hyper_param_KDE = hyper_parameters_KDE[best_hyper_param_ind_KDE]
best_var_KDE = validation_variances_KDE[best_hyper_param_ind_KDE]

validation_variances_sphere = \
            hm.utils.cross_validation(
                    chains_train, \
                    domains_sphere, \
                    hyper_parameters_sphere, \
                    nfold=nfold, \
                    modelClass=hm.model.HyperSphere, \
                    seed=0)

best_hyper_param_ind_sphere = np.argmin(validation_variances_sphere)
best_hyper_param_sphere = hyper_parameters_sphere[best_hyper_param_ind_sphere]
best_var_sphere = validation_variances_sphere[best_hyper_param_ind_sphere]

In [14]:
print("best_var_sphere = {}".format(best_var_sphere))
print("best_var_KDE = {}".format(best_var_KDE))

if best_var_sphere < best_var_KDE:                        
        model = hm.model.HyperSphere(ndim, domains_sphere, 
                                     hyper_parameters=best_hyper_param_sphere)
        print('Using Hypersphere model!')
  
else:                       
        model = hm.model.KernelDensityEstimate(ndim, domains_KDE, 
                                               hyper_parameters=best_hyper_param_KDE)
        print('Using Kernel Density Estimate model!')

best_var_sphere = -8.616605704089338
best_var_KDE = -3.595846079140736
Using Hypersphere model!
